In [196]:
import os 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn

from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from rgf.sklearn import RGFClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

In [197]:
# Loading the train dataset
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
economic_indicators = pd.read_csv('economic_indicators.csv')

# Display the first few rows of the datasets and their shape
display("Train", train.head(), train.shape, "Test", test.head(), test.shape, "Economic Indicators", economic_indicators.head(), economic_indicators.shape)

'Train'

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,target
0,ID_266671248032267278,266671,Kenya,248032,267278,Type_1,8448.0,8448.0,2022-08-30,2022-09-06,7,Repeat Loan,120.85,0.014305,121.0,0
1,ID_248919228515267278,248919,Kenya,228515,267278,Type_1,25895.0,25979.0,2022-07-30,2022-08-06,7,Repeat Loan,7768.50,0.300000,7794.0,0
2,ID_308486370501251804,308486,Kenya,370501,251804,Type_7,6900.0,7142.0,2024-09-06,2024-09-13,7,Repeat Loan,1380.00,0.200000,1428.0,0
3,ID_266004285009267278,266004,Kenya,285009,267278,Type_1,8958.0,9233.0,2022-10-20,2022-10-27,7,Repeat Loan,2687.40,0.300000,2770.0,0
4,ID_253803305312267278,253803,Kenya,305312,267278,Type_1,4564.0,4728.0,2022-11-28,2022-12-05,7,Repeat Loan,1369.20,0.300000,1418.0,0


(68654, 16)

'Test'

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid
0,ID_269404226088267278,269404,Kenya,226088,267278,Type_1,1919.0,1989.0,2022-07-27,2022-08-03,7,Repeat Loan,575.7,0.300000,597.0
1,ID_255356300042267278,255356,Kenya,300042,267278,Type_1,2138.0,2153.0,2022-11-16,2022-11-23,7,Repeat Loan,0.0,0.000000,0.0
2,ID_257026243764267278,257026,Kenya,243764,267278,Type_1,8254.0,8304.0,2022-08-24,2022-08-31,7,Repeat Loan,207.0,0.025079,208.0
3,ID_264617299409267278,264617,Kenya,299409,267278,Type_1,3379.0,3379.0,2022-11-15,2022-11-22,7,Repeat Loan,1013.7,0.300000,1014.0
4,ID_247613296713267278,247613,Kenya,296713,267278,Type_1,120.0,120.0,2022-11-10,2022-11-17,7,Repeat Loan,36.0,0.300000,36.0


(18594, 15)

'Economic Indicators'

,Country,Indicator,YR2001,YR2002,YR2003,YR2004,YR2005,YR2006,YR2007,YR2008,...,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022,YR2023
0,Ghana,"Inflation, consumer prices (annual %)",41.509496,9.360932,29.772980,18.042739,15.438992,11.679184,10.734267,16.494640,...,15.489616,17.149970,17.454635,12.371922,7.808765,7.143640,9.887290,9.971089,31.255895,38.106966
1,Cote d'Ivoire,"Inflation, consumer prices (annual %)",4.361529,3.077265,3.296807,1.457988,3.885830,2.467191,1.892006,6.308528,...,0.448682,1.251500,0.723178,0.685881,0.359409,-1.106863,2.425007,4.091952,5.276167,4.387117
2,Kenya,"Inflation, consumer prices (annual %)",5.738598,1.961308,9.815691,11.624036,10.312778,14.453734,9.758880,26.239817,...,6.878155,6.582154,6.297250,8.005650,4.689806,5.239638,5.405162,6.107936,7.659863,7.671396
3,Ghana,"Official exchange rate (LCU per US$, period av...",0.716305,0.792417,0.866764,0.899495,0.905209,0.915107,0.932619,1.052275,...,2.896575,3.714642,3.909817,4.350533,4.585325,5.217367,5.595708,5.805700,8.272400,11.020408
4,Cote d'Ivoire,"Official exchange rate (LCU per US$, period av...",732.397693,693.713226,579.897426,527.338032,527.258363,522.425625,478.633718,446.000041,...,493.757330,591.211698,592.605615,580.656750,555.446458,585.911013,575.586005,554.530675,623.759701,606.569750


(27, 25)

In [198]:
economic_indicators.loc[((economic_indicators['Country']=='Ghana') & (economic_indicators['Indicator']=='Real interest rate (%)')), ['YR2001', 'YR2002', 'YR2003', 'YR2004',
       'YR2005', 'YR2006', 'YR2007', 'YR2008', 'YR2009', 'YR2010', 'YR2011',
       'YR2012', 'YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018',
       'YR2019', 'YR2020', 'YR2021', 'YR2022', 'YR2023']] =\
           economic_indicators.loc[((economic_indicators['Country']=='Kenya') & (economic_indicators['Indicator']=='Real interest rate (%)'))][['YR2001', 'YR2002', 'YR2003', 'YR2004', \
               'YR2005', 'YR2006', 'YR2007', 'YR2008', 'YR2009', 'YR2010', 'YR2011', 'YR2012', 'YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021',\
                   'YR2022', 'YR2023']].values

In [199]:
economic_indicators.loc[((economic_indicators['Country']=='Ghana') & (economic_indicators['Indicator']=='Lending interest rate (%)')), ['YR2001', 'YR2002', 'YR2003', 'YR2004',
       'YR2005', 'YR2006', 'YR2007', 'YR2008', 'YR2009', 'YR2010', 'YR2011',
       'YR2012', 'YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018',
       'YR2019', 'YR2020', 'YR2021', 'YR2022', 'YR2023']] =\
           economic_indicators.loc[((economic_indicators['Country']=='Kenya') & (economic_indicators['Indicator']=='Lending interest rate (%)'))][['YR2001', 'YR2002', 'YR2003', 'YR2004', \
               'YR2005', 'YR2006', 'YR2007', 'YR2008', 'YR2009', 'YR2010', 'YR2011', 'YR2012', 'YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021',\
                   'YR2022', 'YR2023']].values

In [200]:
economic_indicators.loc[((economic_indicators['Country']=='Ghana') & (economic_indicators['Indicator']=='Interest rate spread (lending rate minus deposit rate, %)')), ['YR2001', 'YR2002', 'YR2003', 'YR2004',
       'YR2005', 'YR2006', 'YR2007', 'YR2008', 'YR2009', 'YR2010', 'YR2011',
       'YR2012', 'YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018',
       'YR2019', 'YR2020', 'YR2021', 'YR2022', 'YR2023']] =\
           economic_indicators.loc[((economic_indicators['Country']=='Kenya') & (economic_indicators['Indicator']=='Interest rate spread (lending rate minus deposit rate, %)'))][['YR2001', 'YR2002', 'YR2003', 'YR2004', \
               'YR2005', 'YR2006', 'YR2007', 'YR2008', 'YR2009', 'YR2010', 'YR2011', 'YR2012', 'YR2013', 'YR2014', 'YR2015', 'YR2016', 'YR2017', 'YR2018', 'YR2019', 'YR2020', 'YR2021',\
                   'YR2022', 'YR2023']].values

In [201]:
economic_indicators = economic_indicators.loc[economic_indicators['Country'].isin(["Kenya", "Ghana"])].melt(id_vars=['Country', 'Indicator'], var_name='Year', value_name='Value')
economic_indicators = economic_indicators.pivot_table(index=['Country', 'Year'], columns='Indicator', values='Value').reset_index()


In [202]:
# Remove dollar signs, percent signs, and parentheses from column names
economic_indicators.columns = economic_indicators.columns.str.replace(r'[\$%(),]', '', regex=True).str.replace(' ', '')


In [203]:
economic_indicators.columns

Index(['Country', 'Year', 'Averageprecipitationindepthmmperyear',
       'Depositinterestrate', 'Fossilfuelenergyconsumptionoftotal',
       'Inflationconsumerpricesannual',
       'Interestratespreadlendingrateminusdepositrate', 'Lendinginterestrate',
       'OfficialexchangerateLCUperUSperiodaverage', 'Realinterestrate',
       'Unemploymentrate'],
      dtype='object', name='Indicator')

In [204]:
economic_indicators.ffill(inplace=True)

In [205]:
train["Year"] = "YR"+pd.to_datetime(train["due_date"]).dt.year.astype(str)
test["Year"] = "YR"+pd.to_datetime(test["due_date"]).dt.year.astype(str)


In [206]:
train = pd.merge(train, economic_indicators, right_on=['Country', 'Year'], left_on=["country_id", "Year"],  how='left')

In [207]:
test = pd.merge(test, economic_indicators, right_on=['Country', 'Year'], left_on=["country_id", "Year"],  how='left')


In [208]:
train.drop(columns=['Year'], inplace=True)
test.drop(columns=['Year'], inplace=True)

In [209]:
train.nunique()

ID                                               68654
customer_id                                       6540
country_id                                           1
tbl_loan_id                                      66520
lender_id                                            4
loan_type                                           22
Total_Amount                                     19076
Total_Amount_to_Repay                            21920
disbursement_date                                  768
due_date                                           893
duration                                            64
New_versus_Repeat                                    2
Amount_Funded_By_Lender                          23391
Lender_portion_Funded                            12844
Lender_portion_to_be_repaid                       9791
target                                               2
Country                                              1
Averageprecipitationindepthmmperyear                 1
Depositint

In [210]:
test.nunique()

ID                                               18594
customer_id                                       4962
country_id                                           2
tbl_loan_id                                      17067
lender_id                                            8
loan_type                                           22
Total_Amount                                      9372
Total_Amount_to_Repay                            10963
disbursement_date                                  656
due_date                                           728
duration                                            50
New_versus_Repeat                                    2
Amount_Funded_By_Lender                           9704
Lender_portion_Funded                             3880
Lender_portion_to_be_repaid                       6782
Country                                              2
Averageprecipitationindepthmmperyear                 2
Depositinterestrate                                  5
Fossilfuel

In [211]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68654 entries, 0 to 68653
Data columns (total 26 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   ID                                             68654 non-null  object 
 1   customer_id                                    68654 non-null  int64  
 2   country_id                                     68654 non-null  object 
 3   tbl_loan_id                                    68654 non-null  int64  
 4   lender_id                                      68654 non-null  int64  
 5   loan_type                                      68654 non-null  object 
 6   Total_Amount                                   68654 non-null  float64
 7   Total_Amount_to_Repay                          68654 non-null  float64
 8   disbursement_date                              68654 non-null  object 
 9   due_date                                       686

In [212]:
[x for x in train.columns if "_id" in x.lower() or x.lower()=="id"]

['ID', 'customer_id', 'country_id', 'tbl_loan_id', 'lender_id']

In [213]:
train["New_versus_Repeat"].value_counts()

New_versus_Repeat
Repeat Loan    68087
New Loan         567
Name: count, dtype: int64

In [214]:
class Preprocessing :
    def __init__(self, train, test) :
        self.train = train 
        self.test = test
        self.train_shape = train.shape
        self.test_shape = test.shape
        self.train_columns = train.columns
        self.test_columns = test.columns
        self.id_cols = [x for x in train.columns if "_id" in x.lower() or x.lower()=="id"] + ["Fossilfuelenergyconsumptionoftotal", "Country", "Averageprecipitationindepthmmperyear"]
        self.target_col = self.train.columns.difference(self.test.columns)
        self.date_cols = [x for x in train.columns if "date" in x.lower()]
        self.cat_cols = [x for x in train.columns if train[x].dtype=="O" and x not in self.id_cols and x not in self.date_cols]
        self.num_cols = [x for x in train.columns if x not in self.cat_cols and x not in self.id_cols and x not in self.target_col and x not in self.date_cols]
        self.test_ids = self.test[self.id_cols]
        
    
    def all_data(self) :
        return pd.concat([self.train, self.test], axis=0)   
    
    def drop_id_cols(self) :
        self.train.drop(columns=self.id_cols, inplace=True)
        self.test.drop(columns=self.id_cols, inplace=True)
        return self.train, self.test
    
    def standardize(self) :
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        self.train[self.num_cols] = scaler.fit_transform(self.train[self.num_cols])
        self.test[self.num_cols] = scaler.transform(self.test[self.num_cols])
        return self.train, self.test
    
    def frequency_encoding_cat_cols(self) :
        for col in self.cat_cols :
            encoding = self.all_data()[col].value_counts()
            self.train[col] = self.train[col].map(encoding)
            self.test[col] = self.test[col].map(encoding)
        return self.train, self.test
    
    def fillna(self) :
        self.train.fillna(0, inplace=True)
        self.test.fillna(0, inplace=True)
        return self.train, self.test
    
    def feature_engineering_num_cols(self) :
        for col in self.num_cols :
            self.train[col+"_squared"] = self.train[col]**2
            self.train[col+"_cubed"] = self.train[col]**3
            self.train[col+"_sqrt"] = np.sqrt(self.train[col])
            self.train[col+"_log"] = np.log(self.train[col]+1)
            
            self.test[col+"_squared"] = self.test[col]**2
            self.test[col+"_cubed"] = self.test[col]**3
            self.test[col+"_sqrt"] = np.sqrt(self.test[col])
            self.test[col+"_log"] = np.log(self.test[col]+1)
        return self.train, self.test
    
    def feature_engineering_date_cols(self) :
        for col in self.date_cols :
            self.train[col+"_year"] = pd.to_datetime(self.train[col]).dt.year
            self.train[col+"_month"] = pd.to_datetime(self.train[col]).dt.month
            self.train[col+"_day"] = pd.to_datetime(self.train[col]).dt.day
            self.train[col+"_dayofweek"] = pd.to_datetime(self.train[col]).dt.dayofweek
            self.train[col+"_weekofyear"] = pd.to_datetime(self.train[col]).dt.isocalendar().week
            
            self.test[col+"_year"] = pd.to_datetime(self.test[col]).dt.year
            self.test[col+"_month"] = pd.to_datetime(self.test[col]).dt.month
            self.test[col+"_day"] = pd.to_datetime(self.test[col]).dt.day
            self.test[col+"_dayofweek"] = pd.to_datetime(self.test[col]).dt.dayofweek
            self.test[col+"_weekofyear"] = pd.to_datetime(self.test[col]).dt.isocalendar().week
            
        self.train.drop(columns=self.date_cols, inplace=True)
        self.test.drop(columns=self.date_cols, inplace=True)
        
        return self.train, self.test
    
    def credit_scoring_feature_engineering(self) :
        # convert dates
        for col in self.date_cols :
            self.train[col] = pd.to_datetime(self.train[col])
            self.test[col] = pd.to_datetime(self.test[col])
        # loan amount ratio
        self.train["loan_to_income"] = self.train["Total_Amount"]/self.train["Total_Amount_to_Repay"]
        self.test["loan_to_income"] = self.test["Total_Amount"]/self.test["Total_Amount_to_Repay"]
        # repayment duration in days
        self.train["repayment_duration"] = (self.train["due_date"] - self.train["disbursement_date"]).dt.days
        self.test["repayment_duration"] = (self.test["due_date"] - self.test["disbursement_date"]).dt.days
        # lender funded ratio
        self.train["lender_funded_ratio"] = self.train["Amount_Funded_By_Lender"]/self.train["Total_Amount"]
        self.test["lender_funded_ratio"] = self.test["Amount_Funded_By_Lender"]/self.test["Total_Amount"]
        
        # loan to duration ratio
        self.train["loan_to_duration_ratio"] = self.train["Total_Amount"]/self.train["repayment_duration"]
        self.test["loan_to_duration_ratio"] = self.test["Total_Amount"]/self.test["repayment_duration"]
        # lender portion ratio
        self.train["lender_portion_ratio"] = self.train["Lender_portion_Funded"]/self.train["Total_Amount"]
        self.test["lender_portion_ratio"] = self.test["Lender_portion_Funded"]/self.test["Total_Amount"]
        # lender repayment ratio
        self.train["lender_repayment_ratio"] = self.train["Lender_portion_to_be_repaid"]/self.train["Total_Amount_to_Repay"]
        self.test["lender_repayment_ratio"] = self.test["Lender_portion_to_be_repaid"]/self.test["Total_Amount_to_Repay"]
        # customer loan count 
        self.train["customer_loan_count"] = self.train.groupby("customer_id")["tbl_loan_id"].transform("count")
        self.test["customer_loan_count"] = self.test.groupby("customer_id")["tbl_loan_id"].transform("count")
        # average loan amount per customer
        self.train["average_loan_amount_per_customer"] = self.train.groupby("customer_id")["Total_Amount"].transform("mean")
        self.test["average_loan_amount_per_customer"] = self.test.groupby("customer_id")["Total_Amount"].transform("mean")
        
        # interest rate
        self.train["interest_rate"] = (self.train['Total_Amount_to_Repay'] - self.train['Total_Amount']) / self.train['Total_Amount']
        self.test["interest_rate"] = (self.test['Total_Amount_to_Repay'] - self.test['Total_Amount']) / self.test['Total_Amount']
        
        # ir mean by lender_id
        self.train["ir_mean_by_lender_id"] = self.train.groupby("lender_id")["interest_rate"].transform("mean")
        self.test["ir_mean_by_lender_id"] = self.test.groupby("lender_id")["interest_rate"].transform("mean")
        
        # ir variance by lender_id
        self.train["ir_variance_by_lender_id"] = self.train.groupby("lender_id")["interest_rate"].transform("var")
        self.test["ir_variance_by_lender_id"] = self.test.groupby("lender_id")["interest_rate"].transform("var")
        
        # ir skewness by lender_id
        self.train["ir_skewness_by_lender_id"] = self.train.groupby("lender_id")["interest_rate"].transform("skew")
        self.test["ir_skewness_by_lender_id"] = self.test.groupby("lender_id")["interest_rate"].transform("skew")
        
        # ir kurtosis by lender_id
        self.train["ir_kurtosis_by_lender_id"] = self.train.groupby("lender_id")["interest_rate"].apply(pd.Series.kurt)
        self.test["ir_kurtosis_by_lender_id"] = self.test.groupby("lender_id")["interest_rate"].apply(pd.Series.kurt)
        
        return self.train, self.test
    
    def feature_engineering(self) :
        # self.feature_engineering_num_cols()
        self.credit_scoring_feature_engineering()
        self.feature_engineering_date_cols()
        return self.train, self.test
    
    def replace_inf(self) :
        self.train.replace([np.inf, -np.inf], np.nan, inplace=True)
        self.test.replace([np.inf, -np.inf], np.nan, inplace=True)
        return self.train, self.test
    
    def preprocess(self) :
        self.feature_engineering()
        self.drop_id_cols()
        self.standardize()
        self.frequency_encoding_cat_cols()
        self.replace_inf()
        self.fillna()
        return self.train, self.test
    
preprocessor = Preprocessing(train, test)
train, test = preprocessor.preprocess()
    

In [215]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68654 entries, 0 to 68653
Data columns (total 39 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   loan_type                                      68654 non-null  int64  
 1   Total_Amount                                   68654 non-null  float64
 2   Total_Amount_to_Repay                          68654 non-null  float64
 3   duration                                       68654 non-null  float64
 4   New_versus_Repeat                              68654 non-null  int64  
 5   Amount_Funded_By_Lender                        68654 non-null  float64
 6   Lender_portion_Funded                          68654 non-null  float64
 7   Lender_portion_to_be_repaid                    68654 non-null  float64
 8   target                                         68654 non-null  int64  
 9   Depositinterestrate                            686

In [216]:
train["due_date_year"].value_counts()

due_date_year
2022    64235
2024     3102
2023     1284
2021       27
2025        6
Name: count, dtype: int64

In [217]:
train["disbursement_date_year"].value_counts()

disbursement_date_year
2022    64405
2024     2970
2023     1240
2021       39
Name: count, dtype: int64

In [218]:
test["disbursement_date_year"].value_counts()

disbursement_date_year
2022    17249
2023      688
2024      629
2021       28
Name: count, dtype: int64

In [219]:
train.target.value_counts()

target
0    67396
1     1258
Name: count, dtype: int64

In [220]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=98)
models = {
    "LogisticRegression" : LogisticRegression(max_iter=1000000),
    "RandomForest" : RandomForestClassifier(),
    "SVC" : SVC(),
    "KNeighbors" : KNeighborsClassifier(),
    "DecisionTree" : DecisionTreeClassifier(),
    "LGBM" : LGBMClassifier(),
    "XGB" : XGBClassifier(),
    "CatBoost" : CatBoostClassifier(),
    "RGF" : RGFClassifier(),
    "GradientBoosting" : GradientBoostingClassifier(),
    "AdaBoost" : AdaBoostClassifier(),
    "ExtraTrees" : ExtraTreesClassifier(),
    "Bagging" : BaggingClassifier(),
    "Voting" : VotingClassifier(estimators=[("lgbm", LGBMClassifier()), ("xgb", XGBClassifier()), ("cat", CatBoostClassifier()),
                                             ("rgf", RGFClassifier()), ("rf", RandomForestClassifier())]),
    "Stacking" : StackingClassifier(estimators=[("lgbm", LGBMClassifier()), ("xgb", XGBClassifier()), ("cat", CatBoostClassifier()),
                                             ("rgf", RGFClassifier()), ("rf", RandomForestClassifier())])
}

In [230]:
# fitting stacking classifier
train_f1_scores = [] 
val_f1_scores = []
val_preds = np.zeros(train.shape[0])
test_preds = np.zeros(test.shape[0])
model="Stacking"
for fold, (train_index, test_index) in enumerate(skf.split(train.drop(columns="target"), train["target"])):
    X_train, X_test = train.drop(columns="target").iloc[train_index], train.drop(columns="target").iloc[test_index]
    y_train, y_test = train["target"].iloc[train_index], train["target"].iloc[test_index]
    models[model].fit(X_train, y_train)
    val_preds[test_index] = models[model].predict(X_test)
    test_preds += models[model].predict(test[X_train.columns])/5
    train_f1_scores.append(f1_score(y_train, models[model].predict(X_train)))
    val_f1_scores.append(f1_score(y_test, models[model].predict(X_test)))
    print(f"Fold {fold} completed")
    print("-"*50)
    print("\n")
    print("Train F1 Score : ", f1_score(y_train, models[model].predict(X_train)), "Validation F1 Score : ", f1_score(y_test, models[model].predict(X_test)))
    print("\n")
print("Mean Train F1 Score : ", np.mean(train_f1_scores), "Mean Validation F1 Score : ", np.mean(val_f1_scores))
print("Standard Deviation Train F1 Score : ", np.std(train_f1_scores), "Standard Deviation Validation F1 Score : ", np.std(val_f1_scores))

[LightGBM] [Info] Number of positive: 1007, number of negative: 53916
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3507
[LightGBM] [Info] Number of data points in the train set: 54923, number of used features: 37
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.018335 -> initscore=-3.980452
[LightGBM] [Info] Start training from score -3.980452
Learning rate set to 0.056991
0:	learn: 0.5136445	total: 28.2ms	remaining: 28.1s
1:	learn: 0.3743266	total: 61.6ms	remaining: 30.7s
2:	learn: 0.2987673	total: 106ms	remaining: 35.3s
3:	learn: 0.2443321	total: 165ms	remaining: 41s
4:	learn: 0.1826304	total: 200ms	remaining: 39.7s
5:	learn: 0.1396802	total: 221ms	remaining: 36.6s
6:	learn: 0.1087892	total: 249ms	remaining: 35.3s
7:	learn: 0.0866200	total: 271ms	remaining: 33.6s
8:	learn: 0.0

In [231]:
sub = pd.DataFrame({"ID" : preprocessor.test_ids["ID"], "target" : test_preds})

In [232]:
sub.target.value_counts()

target
0.0    17872
1.0      611
0.8       38
0.2       29
0.4       24
0.6       20
Name: count, dtype: int64

In [233]:
sub.target = sub.target.apply(lambda x : 1 if x>0.5 else 0)

In [234]:
sub.target.value_counts()

target
0    17925
1      669
Name: count, dtype: int64

In [235]:
sub.to_csv("stacking_ei_sub.csv", index=False)